In [1]:
import numpy as np
import pandas as pd
import json
from utils import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from sklearn import preprocessing

In [2]:
# truth_df = pd.DataFrame(columns=['id','truthMedian','truthClass','truthMean'])
# with open('data/truth.jsonl') as data:
#     for labelobj in data:
#         truth = json.loads(labelobj)
#         truthlabel = {'id': truth['id'], 'truthMedian': truth['truthMedian'], 'truthClass': truth['truthClass'], 'truthMean': truth['truthMean']}
#         truth_df = truth_df.append(truthlabel, ignore_index = True)
# truth_df.head()   

In [3]:
# instances_df = pd.DataFrame(columns=['id','postText'])
# with open('data/instances.jsonl') as data:
# 	for instanceobj in data:
# 		instance = json.loads(instanceobj)
# 		instancerow = {'id': instance['id'], 'postText': instance['postText']}
# 		instances_df = instances_df.append(instancerow, ignore_index=True)
# instances_df.head()

In [4]:
# dataset = instances_df.join(truth_df.set_index('id'), on='id')
# dataset = dataset.drop(labels='id',axis=1)
# for i in range(len(dataset)):
#     dataset['postText'].values[i] = dataset['postText'].values[i][0]
# dataset['postText'].dropna(inplace=True)
# dataset.head()

In [5]:
# # Convert the 'no-clickbait' or 'clickbait' to binary indicator
# dataset['truthClass'] = dataset['truthClass'].apply(classToBinary)
# # Convert floating number in 'truthMedian' column to integer
# dataset['truthMedian'] = dataset['truthMedian'].apply(medianToInteger)
# # Remove all punctuations and clear the text
# dataset['postText'] = dataset['postText'].apply(cleanText)
# dataset.head()

In [6]:
# # length of titles with longest words
# maxLen = maxLengthInPostText(dataset)
# maxLen

In [7]:
# # split the dataset to training and testing set
# train, test = train_test_split(dataset, test_size=0.2)
# X_train, Y_train, Y_train_mean = np.array(train["postText"].tolist()), np.array(train["truthMedian"].tolist()), np.array(train["truthMean"].tolist())
# X_test, Y_test, Y_test_mean = np.array(test["postText"].tolist()), np.array(test["truthClass"].tolist()), np.array(test["truthMean"].tolist())
# print(Y_train.shape)
# print(Y_test.shape)

In [8]:
train_df = pd.DataFrame(columns=['id','postText','truthMedian','truthClass','truthMean'])
with open('data/train.json') as json_data:
    data = json.load(json_data)
    for instance in data:
        train_instance = {'id': instance['text_id'], 'postText': instance['post_text'], 'truthMedian': instance['truth_median'], 'truthClass': instance['click_bait'], 'truthMean': instance['truth_mean']}
        train_df = train_df.append(train_instance, ignore_index = True)
train_df.head()

,id,postText,truthMedian,truthClass,truthMean
0,849630396191055872,"Flying mattress strikes biker at 50 mph, ends ...",0.000000,0,0.200000
1,805003400136036352,The Grand Tour: Jeremy Clarkson is really taun...,0.000000,0,0.200000
2,811134324628226048,Was this murderous attack on a Russian diploma...,0.666667,1,0.533333
3,807681539580502016,"PartyNextDoor brings out Drake, G-Eazy and mor...",0.000000,0,0.266667
4,807465142032232450,A far-right Dutch lawmaker who wants to Make t...,0.000000,0,0.200000


In [9]:
test_df = pd.DataFrame(columns=['id','postText','truthMedian','truthClass','truthMean'])
with open('data/test.json') as json_data:
    data = json.load(json_data)
    for instance in data:
        test_instance = {'id': instance['text_id'], 'postText': instance['post_text'], 'truthMedian': instance['truth_median'], 'truthClass': instance['click_bait'], 'truthMean': instance['truth_mean']}
        test_df = test_df.append(test_instance, ignore_index = True)
test_df.head()

,id,postText,truthMedian,truthClass,truthMean
0,841383755944189952,"John Lewis blasts GOP rep's ""bigoted and racis...",0.000000,0,0.200000
1,852354010820620288,The NBA playoff bracket is now set,0.000000,0,0.200000
2,838728651545903104,Migrants smash up German asylum centre and sev...,0.000000,0,0.133333
3,805465449739718656,"America's most livable states, ranked",0.666667,1,0.666667
4,841713776382627840,WIP or RIP?,1.000000,1,0.933333


In [10]:
# Convert floating number in 'truthMedian' column to integer
train_df['truthMedian'] = train_df['truthMedian'].apply(medianToInteger)
# Remove all punctuations and clear the text
train_df['postText'] = train_df['postText'].apply(cleanText)
train_df.head()

,id,postText,truthMedian,truthClass,truthMean
0,849630396191055872,flying mattress strikes biker at 50 mph ends u...,0,0,0.200000
1,805003400136036352,the grand tour jeremy clarkson is really taunt...,0,0,0.200000
2,811134324628226048,was this murderous attack on a russian diploma...,2,1,0.533333
3,807681539580502016,partynextdoor brings out drake geazy and more ...,0,0,0.266667
4,807465142032232450,a farright dutch lawmaker who wants to make th...,0,0,0.200000


In [11]:
# Convert floating number in 'truthMedian' column to integer
test_df['truthMedian'] = test_df['truthMedian'].apply(medianToInteger)
# Remove all punctuations and clear the text
test_df['postText'] = test_df['postText'].apply(cleanText)
test_df.head()

,id,postText,truthMedian,truthClass,truthMean
0,841383755944189952,john lewis blasts gop reps bigoted and racist ...,0,0,0.200000
1,852354010820620288,the nba playoff bracket is now set,0,0,0.200000
2,838728651545903104,migrants smash up german asylum centre and sev...,0,0,0.133333
3,805465449739718656,americas most livable states ranked,2,1,0.666667
4,841713776382627840,wip or rip,3,1,0.933333


In [12]:
# length of titles with longest words
maxLen = max(maxLengthInPostText(train_df), maxLengthInPostText(test_df))
maxLen

25

In [13]:
X_train, Y_train = np.array(train_df["postText"].tolist()), np.array(train_df["truthMedian"].tolist())
X_test, Y_test = np.array(test_df["postText"].tolist()), np.array(test_df["truthClass"].tolist())
print(X_train[0:10])
print(Y_train[0:10])

['flying mattress strikes biker at 50 mph ends up cushioning his fall by'
 'the grand tour jeremy clarkson is really taunting the bbc now'
 'was this murderous attack on a russian diplomat revenge for turkeys dirty deal with putin'
 'partynextdoor brings out drake geazy and more at summers over concert in la'
 'a farright dutch lawmaker who wants to make the netherlands great again was convicted of inciting discrimination'
 'this prince bought plane tickets for his 80 falcons casual'
 'kristen bell explains to cbs news why her daughters dont watch frozen'
 'the internet reacts predictably to trumps mysterious sweden incident'
 'isis is on the offensive and may have just picked up a surfacetoair missile system'
 'were watching how the markets are responding to trumps']
[0 0 2 0 0 3 0 2 0 1]


In [14]:
# import the glove word embedding file
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B/glove.6B.100d.txt')

In [15]:
Indices = sentences_to_indices(X_train,word_to_index, maxLen)
print("X_Train_indices =\n", Indices.shape)

X_Train_indices =
 (15567, 25)


In [16]:
def ClickBait_BiLSTM(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)   
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    X = Bidirectional(LSTM(128, return_sequences=True))(embeddings)
    # dropout
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences=True))(X)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(128, return_sequences=False))(X)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 2 units
    X = Dense(4)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)  
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices, X) 
    return model

In [17]:
BiLSTM_model = ClickBait_BiLSTM((maxLen,), word_to_vec_map, word_to_index)
BiLSTM_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 25, 100)           40000100  
_________________________________________________________________
bidirectional (Bidirectional (None, 25, 256)           234496    
_________________________________________________________________
dropout (Dropout)            (None, 25, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25, 256)           394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               394240

In [18]:
BiLSTM_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 4)
X_train_indices.shape

(15567, 25)

In [20]:
BiLSTM_model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 32, shuffle=True)

Epoch 1/10
487/487 [==============================] - 21s 33ms/step - loss: 1.0708 - accuracy: 0.5355
Epoch 2/10
487/487 [==============================] - 18s 37ms/step - loss: 0.9549 - accuracy: 0.5724
Epoch 3/10
487/487 [==============================] - 19s 38ms/step - loss: 0.9240 - accuracy: 0.5899
Epoch 4/10
487/487 [==============================] - 20s 41ms/step - loss: 0.8967 - accuracy: 0.6022
Epoch 5/10
487/487 [==============================] - 20s 41ms/step - loss: 0.8752 - accuracy: 0.6032
Epoch 6/10
487/487 [==============================] - 20s 42ms/step - loss: 0.8478 - accuracy: 0.6224
Epoch 7/10
487/487 [==============================] - 21s 44ms/step - loss: 0.8287 - accuracy: 0.6309
Epoch 8/10
487/487 [==============================] - 21s 44ms/step - loss: 0.7929 - accuracy: 0.6477
Epoch 9/10
487/487 [==============================] - 22s 44ms/step - loss: 0.7488 - accuracy: 0.6691
Epoch 10/10
487/487 [==============================] - 21s 44ms/step - loss: 0.686

In [21]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,roc_auc_score, mean_squared_error, classification_report
y_train_pred_oh = BiLSTM_model.predict(X_train_indices)
y_train_pred_binary = onehot_to_binary(y_train_pred_oh)
Y_train_binary = onehot_to_binary(Y_train_oh)

print("LSTM Training Accuracy")
print('Accuracy %s' % accuracy_score(Y_train_binary, y_train_pred_binary))
print('Precision %s' % precision_score(Y_train_binary, y_train_pred_binary))
print('Recall %s' % recall_score(Y_train_binary, y_train_pred_binary))
print('F1 score: %s' % f1_score(Y_train_binary, y_train_pred_binary))
#print('MSE %s' % mean_squared_error(Y_train_mean, y_train_pred_binary))

LSTM Training Accuracy
Accuracy 0.8972184749791225
Precision 0.8568464730290456
Recall 0.6763100436681223
F1 score: 0.7559487492373398


In [22]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
y_pred_onehot = BiLSTM_model.predict(X_test_indices)
y_pred_binary = onehot_to_binary(y_pred_onehot)

print("LSTM Testing Accuracy")
print('Accuracy %s' % accuracy_score(Y_test, y_pred_binary))
print('Precision %s' % precision_score(Y_test, y_pred_binary))
print('Recall %s' % recall_score(Y_test, y_pred_binary))
print('F1 score: %s' % f1_score(Y_test, y_pred_binary))
#print('MSE %s' % mean_squared_error(Y_test_mean, y_pred_binary))

LSTM Testing Accuracy
Accuracy 0.8232271325796505
Precision 0.7398601398601399
Recall 0.513094083414161
F1 score: 0.6059564719358534


In [23]:
print("LSTM Testing report")
print(classification_report(Y_test, y_pred_binary))

LSTM Testing report
              precision    recall  f1-score   support

           0       0.84      0.93      0.89      2861
           1       0.74      0.51      0.61      1031

    accuracy                           0.82      3892
   macro avg       0.79      0.72      0.75      3892
weighted avg       0.81      0.82      0.81      3892



In [24]:
# Save our model
BiLSTM_model.save("BiLSTM_model.h5")

In [25]:
# Write output to test_df
def predict(headline):
    headline = cleanText(headline)
    headline_np = np.array([headline])
    indices = sentences_to_indices(headline_np, word_to_index, max_len = maxLen)
    y_pred_onehot = BiLSTM_model.predict(indices)
    return y_pred_onehot[0][3] + y_pred_onehot[0][2]

In [26]:
test_df['predicted_prob_LSTM'] = test_df['postText'].apply(predict)
test_df.head()

,id,postText,truthMedian,truthClass,truthMean,predicted_prob_LSTM
0,841383755944189952,john lewis blasts gop reps bigoted and racist ...,0,0,0.200000,0.001446
1,852354010820620288,the nba playoff bracket is now set,0,0,0.200000,0.090908
2,838728651545903104,migrants smash up german asylum centre and sev...,0,0,0.133333,0.000018
3,805465449739718656,americas most livable states ranked,2,1,0.666667,0.147155
4,841713776382627840,wip or rip,3,1,0.933333,0.985155


In [42]:
# def ClickBait_BiGRU(input_shape, word_to_vec_map, word_to_index):
#     sentence_indices = Input(input_shape, dtype='int32')
#     embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
#     # Propagate sentence_indices through your embedding layer
#     embeddings = embedding_layer(sentence_indices)   
#     # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
#     X = Bidirectional(GRU(128, return_sequences=True))(embeddings)
#     # dropout
#     X = Dropout(0.5)(X)
#     X = Bidirectional(GRU(128, return_sequences=True))(X)
#     # dropout
#     X = Dropout(0.5)(X)
#     # Propagate X trough another LSTM layer with 128-dimensional hidden state
#     # The returned output should be a single hidden state, not a batch of sequences.
#     X = Bidirectional(GRU(128, return_sequences=False))(X)
#     # dropout
#     X = Dropout(0.5)(X)
#     # Propagate X through a Dense layer with 2 units
#     X = Dense(4)(X)
#     # Add a softmax activation
#     X = Activation('softmax')(X)  
#     # Create Model instance which converts sentence_indices into X.
#     model = Model(sentence_indices, X) 
#     return model

In [43]:
# BiGRU_model = ClickBait_BiGRU((maxLen,), word_to_vec_map, word_to_index)
# BiGRU_model.summary()

In [44]:
# BiGRU_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
# BiGRU_model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 32, shuffle=True)

In [46]:
# # Write output to test_df
# def predictGRU(headline):
#     headline = cleanText(headline)
#     headline_np = np.array([headline])
#     indices = sentences_to_indices(headline_np, word_to_index, max_len = maxLen)
#     y_pred_onehot = BiGRU_model.predict(indices)
#     return y_pred_onehot[0][3] + y_pred_onehot[0][2]

In [47]:
# test_df['predicted_prob_GRU'] = test_df['postText'].apply(predictGRU)
# test_df.head()

In [48]:
# test_df.to_csv('test.csv', index = False, header=True)

In [49]:
# Error Analysis
for i in range(100):
    if Y_test[i] - y_pred_binary[i] != 0:
        print(X_test[i])
        print("Actual Label",Y_test[i])
        print("Prediction Lable",y_pred_binary[i])
        print("Prediction",y_pred_onehot[i])
        print("-------------")

americas most livable states ranked
Actual Label 1
Prediction Lable 0
Prediction [0.29046443 0.5623806  0.12951408 0.01764093]
-------------
driving to the south pole is the coolest road trip youve never heard of
Actual Label 0
Prediction Lable 1
Prediction [0.01627927 0.15102689 0.55517805 0.27751577]
-------------
whats the no 1 ranking worth for the top golfers maybe nothing more than an ego boost
Actual Label 0
Prediction Lable 1
Prediction [0.03212813 0.11661927 0.33967736 0.5115752 ]
-------------
cannabis users are four times more likely to suffer a heart attack
Actual Label 0
Prediction Lable 1
Prediction [0.02853101 0.187033   0.40424696 0.380189  ]
-------------
this transcript of donald trumps victory rally should terrify anyone with a reading age over 12
Actual Label 1
Prediction Lable 0
Prediction [8.7698746e-01 1.2107683e-01 1.8473874e-03 8.8239176e-05]
-------------
selma hayeks dogs  ellen 
Actual Label 0
Prediction Lable 1
Prediction [0.0919378  0.26537797 0.44782224 0

In [50]:
headline = "Here are 10 things you may not know"
predict(headline)

0.7755324

In [51]:
# predictGRU(headline)

In [54]:
test_df.to_json('test.json',orient='records')